# Prepare Xpress Data

In [22]:
import pandas as pd                             # data analysis library
import openpyxl                                 # Read excel data
import os

In [2]:
def timedelta_startdate(row, AD):
    if AD == 'arr':
        col = row['DATETIME_x']
    elif AD == 'dep':
        col = row['DATETIME_y']
    delta = col - start_date
    return delta.days*24*60 + delta.seconds//60

In [3]:
start_date_str = '08/04/2018'
end_date_str   = '08/04/2018'

# Convert strings to dates
start_date = pd.to_datetime(start_date_str, format='%d/%m/%Y')
end_date   = pd.to_datetime(end_date_str,   format='%d/%m/%Y')

### Read linked flights

In [23]:
data_path = os.path.join(os.getcwd(), 'output', 'turnarounds.csv')
linked = pd.read_csv(data_path)

# Convert datetimes to datetime format (currently string)
linked.DATETIME_x = pd.to_datetime(linked.DATETIME_x)
linked.DATETIME_y = pd.to_datetime(linked.DATETIME_y)

### Read stand data from excel file

In [24]:
# Open excel file
data_path = os.path.join(os.getcwd(), 'data', 'standAllocationMaster.xlsx')
wb = openpyxl.load_workbook(data_path)

# Save first sheet with stand data
standsSheet = wb['Stands']

# Read stands from column D in stand sheet
stands = []
standGroups = []

for row in range(3,200):
    thisStand      = standsSheet.cell(row=row, column=4).value
    thisStandGroup = standsSheet.cell(row=row, column=2).value
    
    if thisStand == None: break
    stands.append(thisStand)
    standGroups.append(thisStandGroup)

### Export flight data

In [25]:
#Convert arrival and departure times to total minutes with respect to the start date of the stand plan (midnight)
arr_timeref = linked.apply(lambda row: timedelta_startdate(row, 'arr'), axis=1)
dep_timeref = linked.apply(lambda row: timedelta_startdate(row, 'dep'), axis=1)

buckets = ((end_date - start_date).days + 1) * 24 * 60 // 5

data_path = os.path.join(os.getcwd(), 'data', 'flightData.txt')

with open(data_path,"w+") as f:
    f.write('numBuckets '   +  str(buckets))
    f.write('\nnumFlights ' + str(linked.shape[0]))
    
    f.write('\narrFlightNo depFlightNo')
    for arr, dep in zip(linked.SERVICE_x, linked.SERVICE_y):
        f.write('\n'+arr + ' ' + str(dep))
        
    f.write('\narrTime depTime')
    for arr, dep in zip(arr_timeref, dep_timeref):
        f.write('\n'+str(arr) + ' ' + str(dep))

### Export airport data

In [10]:
data_path = os.path.join(os.getcwd(), 'data', 'airportData.txt')

with open(data_path,"w+") as f:
    f.write('numStands ' + str(len(stands)))
        
    f.write('\nStandNames')
    for stand in stands:
        f.write('\n'+stand)